In [18]:
import string
import random

def add_row(website_name, username, password):
    import mysql.connector
    
    mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = 'PLACEHOLDER_PASSWORD', database = 'password_manager', use_pure = True)
    
    mycursor = mydb.cursor()

    # Inserting values
    sql = '''INSERT INTO username_and_password (website_name, username, password) VALUES (%s, %s, %s)'''
    val = (website_name, username, password)
    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

    mydb.close()

def create_password():
    all_characters = ""
    all_characters += string.ascii_letters
    all_characters += string.digits
    all_characters += string.punctuation

    # Iterating through banned and all characters and skipping the characters that are 
    # banned while adding the allowed ones
    # to the all_allowed_characters variable
    all_allowed_characters = ""
    length = int(input('What is the upper length of the password requirement: '))
    banned_characters = input('Enter all characters that are not allowed (i.e *!?>): ')
    skip_or_not = False
    for character in all_characters:
        for banned_character in banned_characters:
            if character == banned_character:
                skip_or_not = True
                break
            else:
                skip_or_not = False
        if skip_or_not == True:
            continue
        else:
            all_allowed_characters += character

    password = ""
    for i in range(length):
        password += random.choice(all_allowed_characters)

    return password

    
def update_record(password, website_name):
    import mysql.connector
    
    mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = 'PLACEHOLDER_PASSWORD', database = 'password_manager', use_pure = True)
    
    mycursor = mydb.cursor()

    # Inserting values
    sql = "UPDATE username_and_password SET password = %s WHERE website_name = %s"
    val = (password, website_name)
    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record updated.")

    mydb.close()

def delete_record(website_name):
    import mysql.connector
    
    mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = 'Sceptile28908', database = 'password_manager', use_pure = True)
    
    mycursor = mydb.cursor()

    sql = "DELETE FROM username_and_password WHERE website_name = %s"
    val = website_name
    mycursor.execute(sql, (val,))

    mydb.commit()

    print(mycursor.rowcount, "record deleted.")

    mydb.close()

def view_records():
    import mysql.connector
    
    mydb = mysql.connector.connect(host = 'localhost', user = 'root', password = 'PLACEHOLDER_PASSWORD', database = 'password_manager', use_pure = True)
    
    mycursor = mydb.cursor()

    # Inserting values
    sql = "SELECT * FROM username_and_password"
    mycursor.execute(sql)

    records = mycursor.fetchall()
    if len(records) == 0:
        print('There are no records')
    else:
        for row in records:
            print(row)

    mydb.close()

In [21]:
while True:
    user_choice = input('Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit: ').lower()

    if user_choice == 'a':
        existing_or_new = input('Would you like to enter an (e)xisting password or would you like to (c)reate a new one: ')
        if existing_or_new == 'e':
            quantity_to_insert = int(input('How many entries would you like to add (please enter a whole number): '))
            for i in range(quantity_to_insert):
                website_name = input('Please enter the name of the website where you have created your password in: ')
                website_username = input('Please enter the username you created for the website: ')
                website_existing_password = input('Please enter the existing password: ')
                add_row(website_name, website_username, website_existing_password)
        else:
            website_name = input('Please enter the name of the website where you have created your password in: ')
            website_username = input('Please enter the username you created for the website: ')
            new_password = create_password()
            add_row(website_name, website_username, new_password)
    elif user_choice == 'u':
        old_new = input('Would you like to enter a password (y)ou created or would you like the (p)rogram to create one: ')
        if old_new == 'y':
            website_name = input('Enter the name of the website whose record will be changed: ')
            new_password = input('Enter new password: ')
            update_record(new_password, website_name)
        else:
            website_name = input('Enter the name of the website whose record will be changed: ')
            new_password = create_password()
            update_record(new_password, website_name)
    elif user_choice == 'd':
        website_name = input('Enter the website name where you want all of the information deleted')
        delete_record(website_name)
    elif user_choice == 'v':
        view_records()
    else:
        break
    

Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit:  v


There are no records


Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit:  a
Would you like to enter an (e)xisting password or would you like to (c)reate a new one:  e
How many entries would you like to add (please enter a whole number):  2
Please enter the name of the website where you have created your password in:  YouTube
Please enter the username you created for the website:  username_example1
Please enter the existing password:  password_example1


1 record inserted.


Please enter the name of the website where you have created your password in:  Facebook
Please enter the username you created for the website:  username_example2
Please enter the existing password:  password_example2


...

Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit:  d
Enter the website name where you want all of the information deleted Facebook


1 record deleted.


Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit:  v


('YouTube', 'username_example1', "j<hnF{^'PF#1")
('Instagram', 'username_example3', '7k8qGxtYjy?9ps')


Would you like to (a)dd a record, (u)pdate a record, (d)elete a record, (v)iew a record, or (q)uit:  q
